In [ ]:
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout, GlobalAveragePooling2D, RandomFlip, RandomRotation, RandomZoom, RandomContrast,RandomTranslation, RandomHeight, RandomWidth , RandomCrop

parent_dir = './data/chest_xray'
train_dir = parent_dir + '/train'
val_dir = parent_dir + '/val'
test_dir = parent_dir + '/test'

image_height = image_width = 224
image_shape = (image_height, image_width) + (3,)
batch_size = 32
epochs = 30
learning_rate = .0001


def load_images():

    def load_images_for(directory):
        return tf.keras.utils.image_dataset_from_directory(
            directory,
            shuffle=True,
            batch_size=batch_size,
            image_size=(image_height, image_width))

    return [load_images_for(dir) for dir in [train_dir, val_dir, test_dir]]


def print_train_dataset_details(dataset):
    num_classes = len(dataset.class_names)
    print(f'in the training set {num_classes} class names to classify the images for: {dataset.class_names}')
    for image_batch, labels_batch in dataset:
        print(f'image batch shape: {image_batch.shape}')
        print(f'image label shape: {labels_batch.shape}')
        break


def use_buffered_prefetching(train, val, test):

    def use_buffered_prefetching_for(dataset):
        return dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

    return [use_buffered_prefetching_for(dataset) for dataset in [train, val, test]]


def build_transfer_learning_model():
    data_augmentation = Sequential([
        RandomFlip('horizontal'),
        RandomZoom(.2),
        RandomContrast(.2),
        RandomTranslation(.2, .2),
        RandomHeight(.2),
        RandomWidth(.2),
        RandomCrop(224, 224),
        RandomRotation(0.2)
    ])

    preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
    base_model = tf.keras.applications.MobileNetV2(input_shape=image_shape, include_top=False, weights='imagenet')
    base_model.trainable = False

    inputs = Input(shape=image_shape)
    x = data_augmentation(inputs)
    x = preprocess_input(x)
    x = base_model(x, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(.2)(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(32, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(8, activation='relu')(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs, name='pneumonia')
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                           tf.keras.metrics.Precision(name='precision'),
                           tf.keras.metrics.Recall(name='recall')])

    return model



def train_model(m, train, val, test):
    hist = m.fit(train, epochs=epochs, validation_data=val)
    loss, accuracy, precision, recall = m.evaluate(test)
    print(f'the accuracy/precision/recall scores on the test dataset: {accuracy:.3f}/{precision:.3f}/{recall:.3f} with a loss of: {loss:.3f}')

    return hist


def plot_learning_curves(hist):
    acc = hist.history['accuracy']
    val_acc = hist.history['val_accuracy']

    precision = hist.history['precision']
    val_precision = hist.history['val_precision']

    recall = hist.history['recall']
    val_recall = hist.history['val_recall']

    loss = hist.history['loss']
    val_loss = hist.history['val_loss']

    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')

    plt.plot(precision, label='Training Precision')
    plt.plot(val_precision, label='Validation Precision')

    plt.plot(recall, label='Training Recall')
    plt.plot(val_recall, label='Validation Recall')

    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()), 1])
    plt.title('Training and Validation Accuracy, Precision and Recall')

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Binary Cross Entropy')
    plt.ylim([0, 1.0])
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()


train_ds, val_ds, test_ds = load_images()
print_train_dataset_details(train_ds)
train_ds, val_ds, test_ds = use_buffered_prefetching(train_ds, val_ds, test_ds)
model = build_transfer_learning_model()
history = train_model(model, train_ds, val_ds, test_ds)
plot_learning_curves(history)


2023-04-12 15:33:26.487846: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-12 15:33:26.489532: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-12 15:33:26.522096: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-12 15:33:26.523042: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-12 15:33:27.288208: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Found 5216 files belonging to 2 classes.


2023-04-12 15:33:29.198362: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-12 15:33:29.198704: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Found 16 files belonging to 2 classes.
Found 624 files belonging to 2 classes.
in the training set 2 class names to classify the images for: ['NORMAL', 'PNEUMONIA']


2023-04-12 15:33:29.618635: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [5216]
	 [[{{node Placeholder/_0}}]]
2023-04-12 15:33:29.618833: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [5216]
	 [[{{node Placeholder/_0}}]]


image batch shape: (32, 224, 224, 3)
image label shape: (32,)
Epoch 1/30


2023-04-12 15:33:32.130827: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [5216]
	 [[{{node Placeholder/_0}}]]
2023-04-12 15:33:32.131061: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [5216]
	 [[{{node Placeholder/_0}}]]


163/163 [==============================] - ETA: 0s - loss: 0.6471 - accuracy: 0.4852 - precision: 0.9501 - recall: 0.3241

2023-04-12 15:34:47.222303: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [16]
	 [[{{node Placeholder/_4}}]]
2023-04-12 15:34:47.222777: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [16]
	 [[{{node Placeholder/_0}}]]


163/163 [==============================] - 76s 444ms/step - loss: 0.6471 - accuracy: 0.4852 - precision: 0.9501 - recall: 0.3241 - val_loss: 0.5320 - val_accuracy: 0.6875 - val_precision: 1.0000 - val_recall: 0.3750
Epoch 2/30
163/163 [==============================] - 65s 399ms/step - loss: 0.4731 - accuracy: 0.6975 - precision: 0.9709 - recall: 0.6111 - val_loss: 0.4335 - val_accuracy: 0.9375 - val_precision: 1.0000 - val_recall: 0.8750
Epoch 3/30
163/163 [==============================] - 65s 401ms/step - loss: 0.3728 - accuracy: 0.7958 - precision: 0.9696 - recall: 0.7486 - val_loss: 0.4166 - val_accuracy: 0.8750 - val_precision: 0.8000 - val_recall: 1.0000
Epoch 4/30
163/163 [==============================] - 63s 389ms/step - loss: 0.3267 - accuracy: 0.8459 - precision: 0.9629 - recall: 0.8243 - val_loss: 0.3700 - val_accuracy: 0.8750 - val_precision: 0.8000 - val_recall: 1.0000
Epoch 5/30
163/163 [==============================] - 62s 383ms/step - loss: 0.2939 - accuracy: 0.8679 